# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#


# Get scroll height.
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break
    else:
        print('網頁更新中...')
        last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 超大方！信紘科每股賺1.15元　發現金股利2元

▲信紘科經營團隊。（圖／資料照）
記者周康玉／台北報導
半導體設備廠信紘科（6667）公布去年每股獲利1.15元，信紘科表示，考量公司財務體質健全、且資本公積充沛，將營運成果回饋全體股東，擬配發現金股利2元，現金配發率達1.74倍，依3月27日收盤價格57.1元計算，現金殖利率為3.5%，今日早盤最高來到58.8元，
信紘科去（2019）年合併營收為12.45億元，因策略性接單，帶動大型擴建廠案件的營收比重攀升，惟因大型擴建廠案件毛利率低於平均水準，使營業毛利1.77億元、年減38%；全年毛利率14.17%，較前年掉了10.2個百分點，來到低檔。
信紘科表示，去年因中科晶圓廠與南科先進製程廠的建置及擴充，廠務業務年增28.48％，由於施工進度符合預期，旗下化學供應系統、製程特殊廢液回收減量設備的廠務系統設備需求逐步增溫；加上穩定施作拆移機服務工程認列公司，推進2019年下半年整體毛利率、營業利益率皆回升至16％、4％。
目前信紘科已於去年第三季正式推出的全新製程機能水設備，應用於先進製程，在矽晶圓片經蝕刻、黃光、EUV曝光後，使用機能水進行清洗，此設備獲得國際半導體大廠的Golden認證通過，有助拉升綠色製程解決方案的營收比重，增添信紘科未來整體營運穩健的成長表現。
----------------------------------------------------------------------

[1] 新興亞股流出又見緩　台股選股可留意「防疫概念股」

▲疫情之下金融市場劇烈波動，法人建議，以選股為上策，挑選具穩健長期需求，以及價值遭低估的個股為主。（示意圖／記者湯興漢攝）
記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情在各國擴散，全球金融市場動盪不安，投資人關注後續布局方向，中信投信表示，美國祭出貨幣寬鬆政策，對全球股市具有一定的支撐力，盤面上防疫概念股仍是焦點，另外食品、遠距電子類股也有買盤敲進。
根據彭博統計，外資資金流向至3月28日，新興亞股近一周流出幅度明顯縮窄，其中以南韓遭外資賣超最多，金額為18.6億美元（約新台幣567億元）；其次

金鼠年才一開始，全球就遇到新型冠狀病毒的疫情衝擊，包括房市都相當擔心焦慮。不過，因2019年最新的買賣移轉棟數為30萬275棟，代表房市已落底，亦即我在2016年所預估「緩跌4年，盤整4年」的緩跌4年結束，2020年開始將進入盤整階段。
在盤整階段，因為市場仍處於剛性自用及換屋的需求，無法吸引投資，更沒辦法熱到讓投機客買盤介入，所以未來的4年也不可能看到房價有大幅的攀升，也不至於爆大量。
若有人期待第1季房市能爆量，那得仿效百貨公司的「福袋策略」，業主方只要願意降一點價，以鼓勵自用型買盤進入，可望使交易量放大。但若有人冀望房價會因疫情而重挫2、3成，這種如同股市中的投機心態，易錯估房市走勢。
我認為，即使疫情擴大，至多讓自用型買盤遞延到第2季，因為自住買房的需求不太會在乎黑天鵝是大或小隻；若疫情能在第1季控制住，就仍將是剛性需求撐市，也就是要看個案表現，以及從價格、交易量到買氣都是盤整的局面。
由於在自用型剛性需求中，也包括了回流的台商，設廠所需的工業土地、自用的辦公室及住家，對買盤跟買氣都有一定支持度，是我今年看好的產品。只是，目前中南部房價反映的是土地成本跟營造成本，缺工使得工資、土地和建材等成本微幅上漲，繼而帶動新建案房價調升10%，但剛性自用的買盤又無法接受，使得銷售期拉長。
至於其他預售市場的房子，若賣到接近新成屋的價格，就會使得新成屋房價要下降至接近中古屋的房價，那麼中古屋就很難再堅持要賣和新成屋一樣的價錢，除非地段、產品非常好，否則無法獲得自用型剛性買盤的青睞。若賣方保持任性，將望屋興嘆，沒有辦法把錢落袋為安。
對於自用型剛性需求的買家，現階段較易在屬意的地段找到房子，若見價格合理就應該進場。想找到房市的相對低檔進場，我認為目前正是一個時機，祝大家發財！
----------------------------------------------------------------------

[14] 泰山「坐擁金雞母」人人都想搶！　老牌食品廠3策略成功翻身

圖、文／鏡週刊
近年來老牌食品廠泰山備受各界關注，成為各路人馬爭奪的對象，到底泰山為什麼一夕間變得炙手可熱？全因為公司轉投資全家便利商店超過2成。
各路人馬搶奪泰山，看上的是公司持有全家便利商店超過2成的股權，由於全家市占率第2，近年積極異業結合，發展藥局、外食、生鮮超市等複合店，加上布局鮮

▲海力雅集成美學潔能牆獲德國紅點設計大獎。（圖／翻攝官網）
記者姚惠茹／台北報導
台灣新創「海力雅集成」今（30）日宣布，以世界首創的擬真發電建材「美學潔能牆」，在全球60個國家、超過6500件作品中脫穎而出，一舉奪得最高榮譽的紅點設計大獎2020最佳設計獎 (Best of the Best)。
海力雅集成表示，「美學潔能牆」能在競爭激烈的歐洲大獎中嶄露鋒芒，主要是因為建築整合再生能源提供前所未有的解決方案，為減緩地球暖化帶來的衝擊，歐盟提出邁向2050近零排碳的目標，各國也紛紛立法改善建築的能耗規範，並將再生能源納入建築的必要元素之一。
海力雅集成指出，歐盟在過去6年已編列超過30億歐元的預算用在對抗氣候變遷議題，其中就包括整合歐洲太陽能及建築產業專家，然而目前的建築整合太陽能的方式，大多都是在屋頂架設矽晶太陽能板，或是將傳統太陽能板裝設在建築外牆上，對於重視外觀美感的歐洲人來說，接受度不高。
海力雅集成分享，團隊預見市場商機，整合台灣產業優勢，跨面板業、太陽能產業、建築美學及國際顧問人才，充分發揮台灣極致的工藝技術，打造全球首創擬真發電建材「美學潔能牆」，結合設計美學及完善的功能性，成功將太陽能與建材結合，達到減碳的目的又兼具美感及多機能性。
海力雅集成總經理黃庭輝表示，美學潔能牆是台灣團隊自主研發的創新產品，整合美學、安全及綠能等多項幕牆所需的建築特性，並通過歐盟建築法認證及取得各國發明專利，並在2019年於荷蘭完成全球首座太陽能擬真大理石幕牆，如今再獲德國紅點設計大獎產品設計最高榮譽的肯定。
黃庭輝指出，歐洲每年的牆面建材市場規模約5億平方米，蘊含無限的市場潛力，海力雅集成將歐洲視為第一個期望攻下的灘頭堡，美學潔能牆在2019年首次公開於德國慕尼黑最大的國際建築建材展BAU，展場的情境陳列讓參觀的建築師彷彿置身於歐洲美術館。
黃庭輝進一步指出，美學潔能牆讓所有買主為之驚艷，迄今已成功開發荷蘭、比利時、德國等歐洲多國通路，並獲得數十個建案設計洽商，成功進軍歐洲市場的商業模式未來可望複製到美國、中國大陸、中東、日本及台灣市場。
海力雅集成設計團隊將在6月22日受邀前往德國紅點大會上台公開受獎，得獎作品「美學潔能牆」也將獲展示並收藏於德國埃森的紅點博物館，以台灣創新設計為歐洲及全球零能耗建築的減碳目標提供最佳的解決方案，也成功打造零能耗建築與再生能源的產業價值鏈

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（30）日公布2019年營收5.34兆元，年增0.93%；全年稅後純益1153.09億元，年減10.66%；每股稅後純益（EPS）8.32元，較2018年EPS 8.03元增加，連續6年賺逾8元，並公告將在6月23日召開股東會。
鴻海公布去年第4季營收1.74兆元，季增25.45%，年減3.81%；毛利率6.47%，季增0.46個百分點，年減0.54個百分點；稅後純益477.66億元，季增55.77%，年減 23.72%；淨利率 2.74%，季增0.53個百分點，年減0.72個百分點，每股稅後純益（EPS）3.45元。
鴻海公布去年營收5.34兆元，年增0.93%；毛利率5.91%，年減0.36個百分點；稅後純益1153.09億元，年減10.66 %；淨利率2.16%，年減0.28個百分點，每股純益（EPS）8.32元，較2018年EPS 8.03元增加。
鴻海全年營收續創歷史新高，每股純益達8.43元，連續6年賺逾8元，符合市場原先估計鴻海去年每股獲利約在8.3元至8.5元之間的預期，但是全年毛利率、營益率、淨利率繳出三率三降，並同步改寫新低紀錄。
----------------------------------------------------------------------

[38] 中小企業確診肺炎「須暫時關閉場所」！和泰產險推出停業賠償保險　最高賠27萬元

▲多數中小企業執行防疫措施，考量員工遠端工作，或暫時停業。（圖／pixabay，示意圖）
記者楊絡懸／台北報導
因應全球性的新冠肺炎（COVID-19）疫情蔓延，台灣社會上下齊心全力維護國人健康，防疫能力備受世界肯定。和泰產險表示，為了支持中小企業在防疫期間能維持長久經營，率先針對中小企業商家推出「和泰產物停業賠償保險（日額乙型）附加條款」，協助企業主度過難關。
和泰產險指出，為避免疫情擴散，當有確診病例時，確診病患及其接觸者皆須進行隔離，中小企業主將可能因顧客或員工罹患新冠肺炎而受主管機關的命令或防疫措施，使該經營場所須暫時關閉或停業，考量「停業至復工」所需時間往往長達半個月以上，對中小企業主而言無疑是項沉重的負擔。
為了減輕中小企業主的停業損失，和泰產險推出「日額型停業賠償保險附加條款」，如不幸發生法定傳染病

▲南韓總統文在寅。（圖／達志影像／美聯社）
記者謝仁傑／綜合報導
文在寅表示，除收入最高的30％家庭以外，其餘七成家庭都將獲得最高100萬韓元（約新台幣24,700元）的緊急救助款；另外，將在4月份再準備一份額外預算供議會批准，關於中小型企業本月開始將免繳部分保險和水電費。
南韓政府先前的振興措施包括降息，11.7兆韓元（約新台幣2890億元）的補充預算，提高銀行的貨幣遠期頭寸上限，以及為公司提供總計100兆韓元（約新台幣2.47兆元）的救助方案。
▲南韓感染新冠肺炎(COVID-19)的案例逐漸控制。（圖／路透社）
在過去的三週中，韓國每天新的確診人數一直徘徊在100例之內，顯示疫情逐漸有控制住的趨勢，但隨著小規模爆發和進口病例的增加，當局也已加強了邊境檢查。韓國疾病預防控制中心（KCDC）週一報告了78例新病例，其中至少13例是境外移入，目前全國共9,661例確診，死亡人數新增6例來到158人。
KCDC統計，目前又有195人從疫情中康復，康復案例來到5,228例。
南韓政府週日宣布，從4月1日開始，所有海外入境者都必須接受兩週的強制隔離，違反此規定者將可處以一年監禁或最高1,000萬韓元（約新台幣24.7萬元）的罰款。
----------------------------------------------------------------------

[49] 詹宏志找周品均入主美合國際　Wstyle與4大美妝整併

▲Wstyle創辦人周品均入主唯品風尚集團擔任執行長。（圖／唯品風尚提供）
記者洪菱鞠／台北報導
台灣美妝電商集團美合國際實業30日董事會上正式決議，公告電商品牌Wstyle將與美合國際實業旗下原有的86小舖、UNT、BeautyMaker、BeautyEasy等4大美妝品牌跨界整併，集團名稱未來將正式更名為唯品風尚集團，Wstyle創辦人周品均則入主集團擔任執行長。
美合國際實業是由PChome、潤泰集團、華威創投聯手成立的美妝聯盟，董事長詹宏志認為，集團要盡快在產業競爭中突圍，需要一位有深厚電商經驗，又有能力掌握美感與時尚的新生代領導者，因此找來電商女王周品均攜手迎戰時尚電商市場。
周品均曾是東京著衣創辦人，因離婚而離開一手創辦的公司，3年前再度白手起家，創立全新電商服飾品牌Wstyle，卻能在短短3年間年收破億，成為詹宏志找上她的

▲經濟部表示，國內產製的「紅外線額溫槍」禁止出口至三月底，因產能提升出口禁令將不再延長實施。（圖／記者李毓康攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情延燒，額溫槍的需求也跟著增加，經濟部本月初發佈國內「紅外線額溫槍」至三月底禁止出口令，因廠商產能增加，將自四月起開放解禁，並承諾優先供應國內防疫需求。
隨著新冠肺炎疫情持續蔓延，越來越多的機關、場所需要購置額溫槍，經濟部於本月4日宣佈國內「紅外線額溫槍」短暫禁止出口，管制期間自即日起至三月底為止，以優先滿足國內防疫需求。
官員指出，為因應防疫需求，國內紅外線額溫槍廠商加速趕工，產能從原來的三至四千支，預估至四月可望一舉提升至萬支以上，經濟部旋即與國內業者討論後取得共識，將自四月起取消出口禁令。
經濟部表示，出口管制主要是希望讓「紅外線額溫槍」能夠優先給國內使用，經與國內業者取得共識，紅外線額溫槍產能四月起可望一舉突破一萬支，而且產能最高的業者也承諾會優先供應國內需求，方便民眾寄給在國外的親友防疫。
依據「紅外線額溫槍禁止出口」措施，管制的體溫計範圍，包括紅外線非接觸式額溫槍，接觸式額溫槍，耳溫槍，電子體溫計以及紅外線測體溫熱像儀等，但不包含水銀體溫計。
----------------------------------------------------------------------

[60] 俄羅斯能源部副部長：全球原油日需求減少多達2000萬桶

▲全球石油需求下降。（圖／達志影像／美聯社）
記者陳心怡／外電報導
根據《路透社》報導，俄羅斯能源部副部長索羅金（Pavel Sorokin）日前表示，新冠肺炎（COVID-19）的鎖國、封城的政策，已經導緻美國、歐洲和中國大陸的石油需求每日減少約1,500萬至2,000萬桶。
新冠肺炎危機導致油價重挫，再加上石油輸出國組織（OPEC）與包括俄羅斯在內的其他主要產油國的限產協議談判破裂，導致油價崩跌近三分之二。索羅金表示，OPEC在談判失敗後，提高石油產量的計劃帶給市場許多額外的壓力。
索羅金提到，如果油價在兩到三年內保持在每桶25美元，這將造成「大問題」，同時他還表示，沙烏地阿拉伯為了增加市場份額，已經降價出售石油。
索羅金表示，OPEC和其他產油國(OPEC+)仍保持接觸。他重申，OPEC+不能單獨削減石油產量，暗示美國也應該加入減產行動。

▲ 中央銀行總裁楊金龍。（資料照／記者湯興漢攝）
記者李瑞瑾／台北報導
為了減少疫情衝擊，央行針對中小企業推出2000億融通方案，央行總裁楊金龍今（30）日指出，為了請公民營銀行都能幫忙，還親自打電話、寫信給銀行董事長。
楊金龍今日赴立法院針對新冠肺炎疫情對金融市場影響及採取政策進行專案報告，立委針對銀行融通是否受阻一事提問，楊金龍表示，目前銀行資金充裕，來向央行融通的機會比較少。
而面對這次疫情，楊金龍表示，央行除了推出2000億融通方案之外，也是在想有沒有其他辦法支撐銀行業融通。
另外，楊金龍提到，這次為了推出2000億融通方案，自己還親自寫信給公、民營銀行董事長，而且還一一打電話拜託他們，他強調，都是希望銀行端也能幫忙，共度難關。
----------------------------------------------------------------------

[72] 前101董事長林鴻明涉掏空金尚昌重判10年　撤銷發回更審

▲前101董事長林鴻明涉嫌掏空金尚昌案，最高法院撤銷發回更二審。（圖／《ETtoday新聞雲》資料照）
記者吳銘峯／台北報導
前台北101大樓董事長、宏國集團副董事長林鴻明，涉嫌在1998年間利用淡水大筆土地，向中聯信託貸款，並以多重交易的方式，掏空上市公司金尚昌公司，更審遭重判10年。案經上訴第三審，最高法院審理後，認為更審認定的罪名更少、刑度更低，卻定更重的執行刑，量刑違反比例原則，因此撤銷更審10年刑度的判決，發回高等法院更二審。
股票上市公司金尚昌公司原本為三重幫林家私人公司「林三號」，林三號公司後來將公司股票公開發行，改名為金尚昌公司；但金尚昌爆發掏空案件後，2013年間又經股東會同意後，改名為「愛山林」建設公司。
林鴻明原本為金尚昌公司的實際負責人，早年陸續收購家族成員所有的淡水鎮水仙段、米蘭段多筆土地。1998年間，金尚昌以淡水鎮水仙段的69筆土地，向同屬宏國集團的中聯信託貸款18億多元，後來宏國集團爆發財務風暴，這些貸款被中聯信託列呆帳。但中聯信託又將這筆呆帳以12億元轉賣「白天鵝」建設公司，林鴻明再以自己1人支配的「啟揚」公司以12億100萬元向白天鵝買回土地；之後與大隱建設合蓋「藍海」豪宅，賺取10.22億利潤。至於價值12億餘元的米蘭段土地，則在買主王在山等三人以7.9億元取得所有權後，由林鴻明再以5

▲ 新台幣兌美元今天開盤貶值3.4分。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（30）日開盤貶值3.4分、以30.27元兌1美元開出。
新台幣匯率是上周五（27日）收在30.236元兌1美元，今天開盤以30.27元兌1美元開出、貶值3.4分。
截至上午9點05分，新台幣來到30.248元兌1美元，貶值1.2分。
----------------------------------------------------------------------

[84] 台股早盤下跌逾200點！道瓊重挫逾4%　指數失守9500點關卡

▲台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
美股上周五終止連三日反彈，道瓊指數拉回915點，22000點關卡再度失守。台股今（30）日開盤下跌127點、以9571點開出。
（更新）截至上午9點11分，指數來到9423點、下跌266點。
蘋概三王中，台積電（2330）以263.5元開出、下跌3.48%；股王大立光（3008）以3775元開出、下跌2.45%；鴻海（2317）開在69.5元、下跌0.86%。
美國武漢肺炎確診高居全球最多，美股週五（27 日）賣壓沉重，連三個交易日大漲走勢止步，尾盤跌勢加劇，道瓊收跌逾900點。
美股道瓊指數下跌915.39點，或4.06%，收21,636.78點；那斯達克指數下跌295.16點，或3.79 %，收7502.38點；標普500指數下跌88.6點，或3.37%，收2,541.47點；費城半導體指數下跌83.87點，或5.33%，收1,488.75點。
----------------------------------------------------------------------

[85] 美股道瓊下跌915點　台指期開盤跌203點

▲美股道瓊下跌 915.39 點。回頭看台指期今（30）日開盤下跌 203 點，來到 9,376 點。（圖／路透）
記者陳心怡／台北報導
美國新冠肺炎（COVID-19）確診高居全球最多，美股上週五賣壓沉重，連三個交易日大漲走勢止步，尾盤跌勢加劇，道瓊指數下跌逾900點。回頭看台指期今（30）日開盤下跌 203 點，來到 9,376 點。
美股道瓊指數下跌 915.39 點，或 4.06%，收 21,636.78 點

▲口罩實名制2.0第二輪預購結束，今日起至30日可透過網銀、信用卡或ATM轉帳等方式繳款。（示意圖／記者紀佳妘攝）
記者紀佳妘／台北報導
「口罩實名制2.0」第二輪預購結束，今（28）日開始進入付款，民眾可透過網銀、信用卡或ATM轉帳等方式繳款，多家銀行大打防疫優惠戰，持續推出刷卡、數位帳戶加碼回饋，提供3片口罩含運費共22元的全額回饋。
台新銀行加碼舉辦「網路行動銀行口罩實名制2.0全額回饋」活動，民眾至衛福部指定的eMask口罩預購系統或由健保快易通APP使用台新銀行網路或行動銀行成功轉帳支付口罩網購費用，每戶可享限定回饋22元，等於該次買口罩的費用全由台新買單。
中國信託銀行續推「中信罩你」活動，提供刷卡、帳戶付款共2大方案，其中客戶刷中信卡繳費，每戶回饋5元刷卡金，等同3片省1片；另一種則是透過My Way台幣數位帳戶轉帳完成付款，享3片口罩含運費共22元的全額回饋。此外，My Way台幣數位帳戶客戶即日起至4月30日止，參與每一輪口罩線上預購活動以轉帳支付，皆可享一次全額回饋優惠。
為提供客戶更便捷的服務，台北富邦銀行即日起推出「富邦幫你付（富）」限時活動，只要在4月30日前透過北富銀LINE官方帳號，首次開立北富銀數位存款帳戶，並以此帳戶繳付口罩預購費，回饋100%，每人限回饋3次，等同口罩免費送給你，還可享美元定存3個月年息1.88%。
為服務廣大預購民眾線上刷卡需求，永豐銀行特別推出「口罩預購　永豐買單」限時優惠活動，即日起至5月31日，卡友只要刷永豐現金回饋JCB卡在「eMask口罩預購系統」完成線上繳費，可享等值刷卡金回饋，單次22元全額回饋，讓永豐為卡友買單，累計刷愈多次回饋愈多，每月享最高100元刷卡金回饋。
滙豐銀行配合第二輪口罩線上預購，3月25日至27日前至衛福部eMask口罩預購系統預購，並於3月28日至30日刷滙豐信用卡付款，不限卡別、無須登錄，除既有卡片權益外，再享7元刷卡金回饋。
